In [1]:
%matplotlib qt

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from matplotlib.ticker import FuncFormatter

from scipy.stats import linregress
from scipy.ndimage import gaussian_filter1d

import os
import re

from tkinter import Tk
from tkinter.filedialog import askopenfilenames

def find_closest(col, rs):
    dist = abs(col - rs)
    return np.argmin(dist)

import warnings
warnings.filterwarnings('ignore')

## Constants

In [2]:
import scipy.constants as sc

superscript = str.maketrans("-0123456789", "-⁰¹²³⁴⁵⁶⁷⁸⁹")

print('e0:', sc.epsilon_0, sc.unit('vacuum electric permittivity'),
        'e:', sc.elementary_charge, sc.unit('elementary charge'), 
      '\npi:', sc.pi, 
      '\n\u03B5:', sc.epsilon_0, sc.unit('vacuum electric permittivity'), 
      '\nPlank:', sc.hbar, sc.value('reduced Planck constant'), sc.unit('reduced Planck constant'),
      '\nSpeed of light:', sc.c, sc.unit('speed of light in vacuum'),
      '\nBoltzman constant:', sc.physical_constants['Boltzmann constant in eV/K'], sc.unit('Boltzmann constant in eV/K')
     )

W = 8e-6 # channel width
ti = 90e-9 # SiO2 thickness
ds = 4e-10 # GNR thickness
e0 = sc.epsilon_0 
eri = 3.9 # insulator SiO2 dielectric constant
ers = 7 # semiconductor 7AC_GNR dielectric constant
q = sc.elementary_charge
kB = sc.physical_constants['Boltzmann constant in eV/K'][0]
print(e0,q,kB)
ci=eri*e0/ti  # F/m2 
ci

e0: 8.8541878128e-12 F m^-1 e: 1.602176634e-19 C 
pi: 3.141592653589793 
ε: 8.8541878128e-12 F m^-1 
Plank: 1.0545718176461565e-34 1.054571817e-34 J s 
Speed of light: 299792458.0 m s^-1 
Boltzman constant: (8.617333262e-05, 'eV K^-1', 0.0) eV K^-1
8.8541878128e-12 1.602176634e-19 8.617333262e-05


0.00038368147188800006

## Configuration

In [3]:
%matplotlib qt

sweeps = {
    'backward': {'linreg': [True, False], 'ini': 61, 'fin': 182, 'ini_fit': 100,'fin_fit': 120},
    'forward': {'linreg': [True, False], 'ini': 182, 'fin': 303, 'ini_fit': 0,'fin_fit': 4},
    'hysteresis': {'linreg': [False], 'ini': 61, 'fin': 303, 'ini_fit': None,'fin_fit': None}
}

contacts = {
        'A': 'Pd-Pd',
        'B': 'Pd-Pd',
        'C': 'Pd-Pd',
        'D': 'Pd-Pd',
        'E': 'Ti-Ti',
        'F': 'Ti-Ti',
        'G': 'Ti-Pd',
        'H': 'Ti-Pd',
    }

In [20]:
def idvg_fileinfo(filepath):
    filename = os.path.basename(filepath)
    match = re.search(r'^([A-Z])(\d)_(\d+)nm.*_vd_((?:-?[\d\.]+_)+)T_(\d+)K_L_([^_]+)_G_([^_]+)_', filename)
    
    if not match:
        raise ValueError(f"Filename does not match expected format: {filename}")

    dev_letter = match.group(1)
    dev_number = match.group(2)
    device = dev_letter + dev_number
    contact = contacts[dev_letter]

    l = int(match.group(3))
    vds_str = match.group(4)
    vds_values = [float(v) for v in vds_str.strip('_').split('_')]
    vds_values.sort()
    # .strip('_') removes all leading and trailing '_'
    # .split('_') cuts at each '_' and returns the parts in a list
    t = int(match.group(5))
    
    light = match.group(6)
    gas = match.group(7)
    
    return filename, dev_letter, dev_number, device, contact, l, t, vds_values, light, gas

In [5]:
def readexcel_at_vd(data, vd, ini, fin, read_Igs):
    mask = (data['Smu2.V[1][1]'] == vd)
    data_vd = data.loc[mask]
    Vgs = data_vd['Smu1.V[1][1]'][ini:fin].to_numpy()
    Ids = data_vd['Smu2.I[1][1]'][ini:fin].to_numpy().astype(np.float64)
    if read_Igs:
        Igs = data_vd['Smu1.I[1][1]'][ini:fin].to_numpy()
        return Vgs, Ids, Igs
    else:
        return Vgs, Ids

In [10]:
def sci_notation_formatter(x, pos):
    if x == 0:
        return "0"
    elif abs(x) <= 1e-3 or abs(x) >= 1e3:
        return f'{x:.1e}'
    else:
        return f'{x:g}' # g: general format

# FET parameters

### $I_{DS}$ smoothing

In [14]:
def plot_smoothing(Ids, Ids_sm, Vgs, vds_values, vdx, vd, lw, alp, ms):
    cc = np.linspace(0, 1, len(vds_values))
    ccx = cc[vdx]
    col_vd = cm.get_cmap('jet_r')(ccx)
    plt.scatter(Vgs, Ids*1e9, marker='o', s=ms, color=col_vd, alpha=1)
    lab = str(int(vd)) if vd.is_integer() else str(vd)
    plt.plot(Vgs, Ids_sm*1e9, label=lab, linewidth=lw, ls='-', color=col_vd, alpha=alp)

def plot_smoothing_config(filepath, s, fs):
    filename, dev_letter, dev_number, device, contact, l, t, vds_values, light, gas = idvg_fileinfo(filepath)
    plt.title(f'{device} ({contact}) {l}nm {t}K step=0.5V {light} {gas} {s}', fontsize=fs)
    plt.xlabel('$V_{GS}$ (V)', fontsize=fs)
    plt.xlim(-30, 30)
    plt.ylabel('$I_{DS}$ (nA)', fontsize=fs)
    plt.tick_params(labelsize=fs)
    plt.grid()
    if len(vds_values)<4:
        ncol=1
    else:
        ncol=2
    legend = plt.legend(title='$V_{DS}$ (V)', loc='best', fontsize=fs, frameon=True, ncol=2)
    
    fig = plt.gcf()
    fig.set_constrained_layout(True)

In [15]:
# Smoothing function

def plot_smoothed(filepath, sgm, s, lw, al, ms, fs):
    filename, dev_letter, dev_number, device, contact, l, t, vds_values, light, gas = idvg_fileinfo(filepath)
    data = pd.read_excel(filepath)
    s_config = sweeps[s]
    ini, fin = s_config['ini'], s_config['fin']
    
    for vdx, vd in enumerate(vds_values):
        Vgs, Ids = readexcel_at_vd(data, vd, ini, fin, False)
        Ids_sm = gaussian_filter1d(Ids, sigma=sgm)
        plot_smoothing(Ids, Ids_sm, Vgs, vds_values, vdx, vd, lw, al, ms)

    plot_smoothing_config(filepath, s, fs)

In [13]:
# Test

s = 'backward'
sgm = 2
al = 0.7
ms = 1
fs = 14
lw = 1

filepaths = ['data/E1_0500nm_idvg_0_+30_-30_+30_0_0.50_vd_2.5_5_10_-2.5_-5_-10_T_298K_L_DARK_G_VAC_V02_Tsweep.xlsx', 'data/F1_1000nm_idvg_0_+30_-30_+30_0_0.50_vd_5_10_20_-5_-10_-20_T_253K_L_DARK_G_VAC_V01.xlsx', 'data/H1_0500nm_idvg_0_+30_-30_+30_0_0.50_vd_2.5_5_10_-2.5_-5_-10_T_236K_L_DARK_G_VAC_V01.xlsx', 'data/F1_1000nm_idvg_0_+30_-30_+30_0_0.50_vd_5_10_20_-5_-10_-20_T_348K_L_DARK_G_VAC_V01.xlsx']
for filepath in filepaths:
    fig = plt.figure(dpi=300)
    plot_smoothed(filepath, sgm, s, lw, al, ms, fs)
plt.show()

### General

In [16]:
# filepath: filepath that contains the data that you want to use for calculations
# df:
#   - Input 'None': function will return dataframe with values extracted only from the filepath
#   - Input dataframe: dataframe in which to add the output data (new data will be concatenated)
# s: sweep
# sgm: smoothing parameter
# see_smoothing: True to plot smoothing of Ids, False not to
# save_smoothing: True to save smoothing plot, False not to
# fs_sm, lw_sm, al_sm, ms_sm: smoothing plot fontsize, linewidth, alpha and markersize

# -------------------------------------

def FET_params(filepath, df, s, sgm, see_smoothing, save_smoothing, fs_sm, lw_sm, al_sm, ms_sm):
    filename, dev_letter, dev_number, device, contact, l, t, vds_values, light, gas = idvg_fileinfo(filepath)
    data = pd.read_excel(filepath)

    s_config = sweeps[s]
    ini, fin = s_config['ini'], s_config['fin']
    ini_fit, fin_fit = s_config['ini_fit'], s_config['fin_fit']
    
    new_rows = []
    
    if see_smoothing:
        fig = plt.figure(dpi=300)
    
    for vdx, vd in enumerate(vds_values):
        Vgs, Ids = readexcel_at_vd(data, vd, ini, fin, False)
        Ids_sm = gaussian_filter1d(Ids, sigma=sgm)
        Ids_sm_abs = gaussian_filter1d(np.abs(Ids), sigma=sgm)
        
        if see_smoothing:
            plot_smoothing(Ids, Ids_sm, Vgs, vds_values, vdx, vd, lw_sm, al_sm, ms_sm)
            
        Ids_on = np.max(Ids_sm_abs)*1e9 # in nA
        Ids_off = np.min(Ids_sm_abs)*1e9
        ratio = Ids_on/Ids_off
        
        x, y = Vgs[ini_fit:fin_fit], np.sqrt(np.abs(Ids))[ini_fit:fin_fit]
        slope, intercept, *_ = linregress(x,y)
        Vth = -intercept/slope
        
        SS = np.diff(Vgs)/np.diff(np.log10(np.abs(Ids_sm)))
        index_off = np.argmin(Ids_sm_abs)
        index_th = find_closest(Vgs, Vth)
        SS_start, SS_end = sorted([index_th, index_off])
        if SS_start== SS_end:
            SS_min = np.nan
            ''' what I did to check what happened:
            print(filename, Vth)
            fig = plt.figure(dpi=300)
            plot_smoothing(Ids, Ids_sm, Vgs, vds_values, vdx, vd, lw_sm, al_sm, ms_sm)
            plt.axvline(x=Vgs[index_off], color='red', linestyle=':', label='off', linewidth=2, alpha=0.7)
            plt.axvline(x=Vth, color='blue', linestyle='--', label='th', linewidth=2, alpha=0.7)
            plot_smoothing_config(filepath, s, fs_sm)
            '''
        else:
            SS_min = np.min(np.abs(SS[SS_start:SS_end]))
            if SS_min == 0: SS_min = np.nan

        Nit_SS = (((SS_min*np.log10(np.e))/(kB*t))-1)*(ci/q)

        mu_lin_array = (l*1e-9 / (W * ci * vd)) * np.diff(Ids_sm)/np.diff(Vgs)
        mu_max_lin = np.max(mu_lin_array)

        sqrt_Ids = np.sqrt(np.maximum(Ids_sm, 0))
        mu_sat_array = (2 * l*1e-9 / (W * ci)) * (np.diff(sqrt_Ids)/np.diff(Vgs))**2
        mu_max_sat = np.max(mu_sat_array)

        new_row = {
            'Device': device,
            'Light': light,
            'Gas': gas,
            '$L$ (nm)': l,
            '$T$ (K)': t,
            'Sweep': s,
            '$V_{DS}$ (V)': vd,
            '$E_{DS}$ (V/nm)': vd/l,
            '$I_{DS,on}$ (nA)': Ids_on,
            '$I_{DS,off}$ (nA)': Ids_off,
            'ratio': ratio,
            '$V_{th}$ (V)': Vth,
            '$SS_{min}$ (V/decade)': SS_min,
            '$\\mu_{lin}$ (cm$^2$/Vs)': mu_max_lin * 1e4,
            '$\\mu_{sat}$ (cm$^2$/Vs)': mu_max_sat * 1e4,
            '$N_{it,SS}$ (cm$^{-2}$)': Nit_SS*1e-4 if not np.isnan(Nit_SS) else np.nan
        } # 'N_{it,\\Delta} (cm$^{-2}$)' & 'N_{it,diff} (cm$^{-2}$)' rows left empty; data will be added here by following functions

        new_rows.append(new_row)
        
    if see_smoothing:
        plot_smoothing_config(filepath, s, fs_sm)
        if save_smoothing:
            folder = f'figures/Ids_smoothing'
            os.makedirs(folder, exist_ok=True)
            name = f'idvg_smoothed_{light}_{gas}_{device}_{l}nm_{t}K_{s}'
            plt.savefig(f'{folder}/{name}.png', dpi=300, bbox_inches='tight')
            plt.savefig(f'{folder}/{name}.svg', dpi=300, bbox_inches='tight')
        
    df_new_rows = pd.DataFrame(new_rows) # each dictionary in the list becomes a new row
    
    if df is None:
        return df_new_rows
    else:
        df_new = pd.concat([df, df_new_rows], ignore_index=True)   
        return df_new

### $N_{it,\Delta}$

In [17]:
# filepath: filepath that you want to use for calculations
# df: dataframe in which to add the output data

# -------------------------------------

def compute_Nit_deltas(filepath, df):
    filename, dev_letter, dev_number, device, contact, l, t, vds_values, light, gas = idvg_fileinfo(filepath)
    
    mask_f = (
        (df['Device'] == device) &
        (df['Light'] == light) &
        (df['Gas'] == gas) &
        (df['$L$ (nm)'] == l) &
        (df['$T$ (K)'] == t) )
    df_f = df.loc[mask_f]
    
    for vdx, vd in enumerate(vds_values):
        mask_vd = ( (df_f['$V_{DS}$ (V)'] == vd) )
        df_vd = df_f.loc[mask_vd]
        
        df_vd_fw = df_vd[(df_vd['Sweep'] == 'forward')]
        vth_fw = df_vd_fw['$V_{th}$ (V)'].iloc[-1] 
        df_vd_bw = df_vd[(df_vd['Sweep'] == 'backward')]
        vth_bw = df_vd_bw['$V_{th}$ (V)'].iloc[-1] 
        
        diff = abs( vth_fw-vth_bw )
        Nit_delta = (ci*diff)/q
        
        # Assign Nit_delta to corresponding df row (same value for both fw and bw!)
        df.loc[mask_f & mask_vd, '$N_{it,\\Delta}$ (cm$^{-2}$)'] = Nit_delta * 1e-4
        
    return df

### $N_{it,diff}$

In [18]:
# df: dataframe that contains input data and also in which output data will be added

# -------------------------------------

def compute_Nit_diff(df):
    rows = []
    grouped = df.groupby(['Device', 'Light', 'Gas', '$L$ (nm)', 'Sweep', '$V_{DS}$ (V)'])

    for (device, light, gas, l, sweep, vd), group in grouped:
        
        group_sorted = group.sort_values('$T$ (K)')

        T = group_sorted['$T$ (K)'].to_numpy()
        Vth = group_sorted['$V_{th}$ (V)'].to_numpy()

        if len(np.unique(T)) < 2:
            # Not enough T points to compute diff
            continue
            
        for i in range(len(T)-1):
            T1, T2 = T[i], T[i+1]

            if T1 != T2:
                V1, V2 = Vth[i], Vth[i+1]
            else:
                T2 = T[i+2]
                V1, V2 = Vth[i], Vth[i+2]
                
            delta_T = T2 - T1
            delta_V = V2 - V1
            dVth_dT = delta_V / delta_T
            Nit_diff = (ci / (q * kB)) * dVth_dT # falta q ????????????????

            # Assign Nit_diff to corresponding df row, and to T1 (therefore, last T won't have a value assigned!)
            mask = (
                (df['Device'] == device) &
                (df['Light'] == light) &
                (df['Gas'] == gas) &
                (df['$L$ (nm)'] == l) &
                (df['Sweep'] == sweep) &
                (df['$V_{DS}$ (V)'] == vd) &
                (df['$T$ (K)'] == T1) &
                (df['$V_{th}$ (V)'] == V1) # Use Vth (arbitrarily) to distinguish case T1=T2
            )

            df.loc[mask, '$N_{it,diff}$ (cm$^{-2}$)'] = Nit_diff*1e-4
    return df

### Compute

#### All df_FET.xlsx

In [19]:
df_FET_cols = ['Device', 'Light', 'Gas','$L$ (nm)','$T$ (K)', 'Sweep', '$V_{DS}$ (V)', '$E_{DS}$ (V/nm)',
               '$I_{DS,on}$ (nA)', '$I_{DS,off}$ (nA)', 'ratio', '$V_{th}$ (V)',
               '$SS_{min}$ (V/decade)', '$\\mu_{lin}$ (cm$^2$/Vs)', '$\\mu_{sat}$ (cm$^2$/Vs)',
               '$N_{it,SS}$ (cm$^{-2}$)','$N_{it,\\Delta}$ (cm$^{-2}$)','$N_{it,diff}$ (cm$^{-2}$)']
        
df_FET = pd.DataFrame(columns=df_FET_cols)

In [68]:
folder = 'data/'
print('Files skipped because identical file done in T sweep is already there:')
printed = False

for filename in os.listdir(folder):
    if not filename.endswith(".xlsx"):
        continue
    if "298K" in filename and "_Tsweep" not in filename:
        match = re.match(r"(.*)_V0.*\.xlsx", filename)
        base_name = match.group(1)

        # Skip this file if any matching _Tsweep file exists
        skip = False
        for f in os.listdir(folder):
            if f.startswith(base_name) and "_Tsweep" in f and f.endswith(".xlsx"):
                print(filename)
                skip = True
                if not printed:
                    printed = True
                break 
        if skip: continue

    filepath = f'{folder}{filename}'
    
    df_sweeps = []
    for s in list(sweeps.keys())[:2]:
        df_s = FET_params(filepath, None, s, sgm=2, see_smoothing=False, save_smoothing=False, fs_sm=14, lw_sm=1.25, al_sm=0.7, ms_sm=1)
        df_sweeps.append(df_s)
        
    df_s_combined = pd.concat(df_sweeps, ignore_index=True)
    df_new_rows = compute_Nit_deltas(filepath, df_s_combined)
    df_FET = pd.concat([df_FET, df_new_rows], ignore_index=True)   

df_FET = compute_Nit_diff(df_FET)

df_FET.to_excel(f'df_FET.xlsx', index=False)

if not printed: print('None.')

display(df_FET)

Files skipped because identical file done in T sweep is already there:
E1_0500nm_idvg_0_+30_-30_+30_0_0.50_vd_2.5_5_10_-2.5_-5_-10_T_298K_L_DARK_G_VAC_V01.xlsx
E3_0200nm_idvg_0_+30_-30_+30_0_0.50_vd_1_2_4_-1_-2_-4_T_298K_L_DARK_G_VAC_V01.xlsx
F1_1000nm_idvg_0_+30_-30_+30_0_0.50_vd_5_10_20_-5_-10_-20_T_298K_L_DARK_G_VAC_V01.xlsx
G1_0200nm_idvg_0_+30_-30_+30_0_0.50_vd_1_2_4_-1_-2_-4_T_298K_L_DARK_G_VAC_V01.xlsx
G3_1000nm_idvg_0_+30_-30_+30_0_0.50_vd_5_10_20_-5_-10_-20_T_298K_L_DARK_G_VAC_V02_recontacted.xlsx
H1_0500nm_idvg_0_+30_-30_+30_0_0.50_vd_2.5_5_10_-2.5_-5_-10_T_298K_L_DARK_G_VAC_V01.xlsx


,Device,Light,Gas,$L$ (nm),$T$ (K),Sweep,$V_{DS}$ (V),$E_{DS}$ (V/nm),"$I_{DS,on}$ (nA)","$I_{DS,off}$ (nA)",ratio,$V_{th}$ (V),$SS_{min}$ (V/decade),$\mu_{lin}$ (cm$^2$/Vs),$\mu_{sat}$ (cm$^2$/Vs),"$N_{it,SS}$ (cm$^{-2}$)","$N_{it,\Delta}$ (cm$^{-2}$)","$N_{it,diff}$ (cm$^{-2}$)"
0,A2,DARK,VAC,200,234,backward,0.5,0.0025,2.749570,0.009926,277.010807,-14.242694,9.684772,0.000003,0.000018,4.971166e+13,1.860402e+12,-329982880949895.75
1,A2,DARK,VAC,200,234,backward,1.0,0.0050,12.417102,0.059165,209.872578,-12.358766,11.286255,0.000005,0.000063,5.797163e+13,2.047376e+12,-454954884175578.5625
2,A2,DARK,VAC,200,234,backward,2.0,0.0100,78.939942,0.536307,147.191737,-9.434408,13.365132,0.000023,0.000338,6.869385e+13,2.115501e+12,-733140501421912.625
3,A2,DARK,VAC,200,234,backward,4.0,0.0200,680.126278,10.326310,65.863440,-3.726025,20.582734,0.000089,0.001595,1.059201e+14,3.428600e+12,-1143043115744891.25
4,A2,DARK,VAC,200,234,forward,0.5,0.0025,2.502747,0.001536,1629.904176,-22.011359,6.671142,0.000004,0.000038,3.416827e+13,1.860402e+12,-355091103605941.875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1281,H3,DARK,VAC,200,298,forward,2.0,0.0100,0.135996,0.003046,44.653299,-22.483939,0.653934,0.000005,0.000009,2.408959e+12,1.797852e+12,NaN
1282,H3,DARK,VAC,200,298,forward,4.0,0.0200,0.301587,0.004590,65.706685,-24.295494,0.526955,0.000003,0.000031,1.894697e+12,2.160796e+12,NaN
1283,H3,DARK,VAC,200,298,forward,-1.0,-0.0050,0.034425,0.004907,7.014921,-27.110819,3.143869,0.000010,0.000038,1.249321e+13,3.832930e+12,NaN
1284,H3,DARK,VAC,200,298,forward,-2.0,-0.0100,0.038414,0.005602,6.857683,-34.677195,6.444462,0.000004,0.000005,2.586063e+13,6.158986e+12,NaN


#### Add new file to df_FET.xlsx

No need to calculate everything again!

Won't add if already in dataframe. DOESN'T WORK!!! Adds anyway :(

In [59]:
'''

filepaths = askopenfilenames(title="Select file")

folder = 'data/'
df_FET = pd.read_excel('df_FET.xlsx')
print('Files skipped because identical file done in T sweep is already there:')
for filepath in filepaths:
    filename = os.path.basename(filepath)
    if "298K" in filename and "_Tsweep" not in filename:
        match = re.match(r"(.*)_V0.*\.xlsx", filename)
        base_name = match.group(1)
        skip = False
        for f in os.listdir(folder):
            if f.startswith(base_name) and "_Tsweep" in f and f.endswith(".xlsx"):
                print(filename)
                skip = True
                break 
        if skip: continue # Skip this file if any matching _Tsweep file exists
    
    df_sweeps = []
    for s in list(sweeps.keys())[:2]:
        df_s = FET_params(filepath, None, s, sgm=2, see_smoothing=False, save_smoothing=False, fs_sm=14, lw_sm=1.25, al_sm=0.7, ms_sm=1)
        df_sweeps.append(df_s)
        
    df_s_combined = pd.concat(df_sweeps, ignore_index=True)
    df_new_rows = compute_Nit_deltas(filepath, df_s_combined)
    df_FET = pd.concat([df_FET, df_new_rows], ignore_index=True)   

df_FET = compute_Nit_diff(df_FET)

# Show duplicates to inspect
print("Duplicate rows:")
print(df_FET[df_FET.duplicated(keep=False)])

df_FET.drop_duplicates(ignore_index=True, keep='first', inplace=True)
#df_FET = df_FET[~df_FET.duplicated(keep='first')].reset_index(drop=True)

df_FET.to_excel(f'df_FET.xlsx', index=False)

display(df_FET)

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-59-12f9d343f8cd>, line 41)

# Figures

### $RW$

#### $RW$ vs $L$

In [21]:
def extract_Lnm_value(filename):
    match = re.search(r'(\d+)nm', filename)
    return int(match.group(1))

def rw(df_rw_filepath, filepaths, s, vds_target, vgs_vals, plot_linreg, fs, lw, ms, colormap, save, compute, save_compute, findclosest_vds):
    s_config = sweeps[s]
    ini, fin = s_config['ini'], s_config['fin']
    ini_fit, fin_fit = s_config['ini_fit'], s_config['fin_fit']

    filepaths = list(filepaths)
    filepaths.sort(key=extract_Lnm_value)
    
    rows = []
    for fx, filepath in enumerate(filepaths):
        if fx==0:
            filename, dev_letter, dev_number, device, contact, l, t, vds_values, light, gas = idvg_fileinfo(filepath)
        else:
            filename, dev_letter, dev_number, device, contact, l, t_fx, vds_values, light, gas = idvg_fileinfo(filepath)
            if t != t_fx:
                print('Error: different T.')
                return

        if findclosest_vds:
            vds_idx = find_closest(np.array(vds_values), vds_target)
            vds = vds_values[vds_idx]
        else:
            if vds_target not in vds_values:
                print('Vds target not in file Vds values.')
                return
            vds = vds_target

        data = pd.read_excel(filepath)
        Vgs, Ids = readexcel_at_vd(data, vds, ini, fin, read_Igs=False)
        if compute:
            x_linreg_idvg, y_linreg_idvg = Vgs[ini_fit:fin_fit], np.sqrt(np.abs(Ids))[ini_fit:fin_fit]
            slope_idvg, intercept_idvg = np.polyfit(x_linreg_idvg, y_linreg_idvg, 1)
            vth = -intercept_idvg/slope_idvg
                
        for vgs in vgs_vals:
            index = np.where(Vgs == vgs)[0][0]
            ids = Ids[index]
            rw = W*vds/ids
            new_row = {
                'L': l,
                'RW': rw,
                'Vgs': vgs
            }
            rows.append(new_row)
                
    df_RW = pd.DataFrame(rows)
    
    # Plot
    
    cc = np.linspace(0, 1, len(vgs_vals))
    if isinstance(colormap, str):
        if colormap == 'Default':
            colors = [cm.jet(x) for x in cc][::-1]
        else:
            cmap = getattr(cm, colormap)
            colors = [cmap(x) for x in cc][::-1]
    else:
        colors = [colormap(x) for x in cc][::-1]
        
    mins, maxs = [], []
    newdf_rows = []
    
    for vgsx, vgs in enumerate(vgs_vals):
        df_RW_vgs = df_RW[df_RW['Vgs'] == vgs]
        x_vals = df_RW_vgs['L'].tolist()
        y_vals_Ohms = df_RW_vgs['RW'].tolist() # in Ohms
        y_vals = df_RW_vgs['RW']*1e-6 # in MOhms
        y_vals = y_vals.tolist()
        plot_label = str(int(vgs)) if (isinstance(vgs, float) and vgs.is_integer()) else str(vgs)
        plt.plot(x_vals, y_vals, label=plot_label, linewidth=lw, ls='-', marker='o', ms=ms, color=colors[vgsx])
        mins.append(min(y_vals))
        maxs.append(max(y_vals))
        
        slope, intercept = np.polyfit(x_vals, y_vals_Ohms, 1)
        if plot_linreg: 
            RW_fit = (slope * np.array(x_vals) + intercept)*1e-6 # in MOhms
            plt.plot(x_vals, RW_fit, color=colors[vgsx], ls='--', lw=lw)
        
        if compute:
            mu_RW = 1*1e4/(ci*(vgs-vth)*slope*1e9)
            # *1e4 for m^2-->cm^2
            # slope*1e9 for slope /nm-->/m
            Rc = intercept/W
            new_row = {
                'Contact': contact,
                'Light': light,
                'Gas': gas,
                '$T$ (K)': t,
                'Sweep': s,
                '$V_{DS}$ (V)': vds_target,
                '$V_{GS}$ (V)': vgs,
                '$\mu_{RW}$ (cm$^2$/Vs)': mu_RW,
                '$R_c$ ($\Omega$)': Rc,
                '$R_cW$ (k$\Omega$$\mu$m)': Rc*W*1e-3*1e6 # *1e-3 for Ohms-->kOhms; *1e6 for m-->microm
            }
            newdf_rows.append(new_row)
                
    plt.title(f'{contact} {t}K $V_{{DS}}$={vds_target}V {light} {gas} {s}', fontsize=fs)
    
    if len(vgs_vals)<4:
        ncol=1
    else:
        ncol=2
    plt.legend(title='$V_{GS}$ (V)', loc='best', fontsize=fs, frameon=True, ncol=ncol)
    
    plt.xlabel('$L$ (nm)', fontsize=fs)
    plt.xlim(df_RW['L'].min(), df_RW['L'].max())
    plt.ylabel('$RW$ (M$\Omega$m)', fontsize=fs)
    ymin, ymax = min(mins), max(maxs)
    # ymin, ymax = 0.008918336945416855, 4.60490759444836
    plt.ylim(ymin, ymax)
    
    plt.tick_params(labelsize=fs)
    plt.grid()
    plt.gca().yaxis.set_major_formatter(FuncFormatter(sci_notation_formatter))
    
    plt.tight_layout()
    
    if save:
        folder = f'figures/RW_vs_L'
        # folder = f'../../../Presentations/figures/Parameters'
        os.makedirs(folder, exist_ok=True)
        
        name = f'RWvsL_{contact}_{t}K_vds_{vds_target}V_{light}_{gas}_{s}'
        if plot_linreg: name += '_linreg'

        plt.savefig(f'{folder}/{name}.png', dpi=300, bbox_inches='tight')
        plt.savefig(f'{folder}/{name}.svg', dpi=300, bbox_inches='tight')
    
    if compute:
        newdf = pd.DataFrame(newdf_rows)
        if save_compute:
            if os.path.exists(df_rw_filepath):
                df_rw = pd.read_excel(df_rw_filepath)
            else:
                df_rw = pd.DataFrame(columns=['Contact', 'Light', 'Gas', '$T$ (K)', 'Sweep', '$V_{DS}$ (V)', '$V_{GS}$ (V)', '$\mu_{RW}$ (cm$^2$/Vs)','$R_c$ ($\Omega$)','$R_cW$ (k$\Omega$$\mu$m)'])

            df_output = pd.concat([df_rw, newdf])

            # Drop duplicates based on the key columns, keeping the last (i.e., from new_rows)
            df_output = df_output.drop_duplicates(subset=['Contact', 'Light', 'Gas', '$T$ (K)', 'Sweep', '$V_{DS}$ (V)', '$V_{GS}$ (V)'], keep='last')
            # Sort
            df_output = df_output.sort_values(by=['Contact', 'Light', 'Gas', '$T$ (K)', 'Sweep', '$V_{DS}$ (V)', '$V_{GS}$ (V)']).reset_index(drop=True)

            df_output.to_excel(df_rw_filepath, index=False)
        display(newdf)
    
    return ymin, ymax

ymin, ymax:

Ti-Ti: 0.008918336945416855 4.297327888712311

Ti-Pd: 0.023224726566263898 4.60490759444836

In [22]:
# Test

filepaths = askopenfilenames(title="Select file")

df_rw_filepath = 'df_Rc_muRW.xlsx'
s = 'backward'
vds_targets = [1,2,4]
vgs_vals = [-30,-20,-10]
plot_linreg = True
fs = 14
lw = 1
ms = 5
colormap = 'jet'
save = True
compute = True
save_compute = True
findclosest_vds = False

ymins, ymaxs = [], []

for vds_target in vds_targets:
    fig = plt.figure(dpi=300)
    ymin, ymax = rw(df_rw_filepath, filepaths, s, vds_target, vgs_vals, plot_linreg, fs, lw, ms, colormap, save, compute, save_compute, findclosest_vds)
    ymins.append(ymin)
    ymaxs.append(ymax)
    
print(min(ymins), max(ymaxs))
plt.show()

,Contact,Light,Gas,$T$ (K),Sweep,$V_{DS}$ (V),$V_{GS}$ (V),$\mu_{RW}$ (cm$^2$/Vs),$R_c$ ($\Omega$),$R_cW$ (k$\Omega$$\mu$m)
0,Ti-Ti,DARK,VAC,298,backward,4,-30,-0.000048,-1.264562e+08,-1.011650e+06


0.008918336945416855 0.035530236737069426


#### $RW$ parameters vs $V_{DS}$

In [23]:
def plot_RW_param_vs_Vds(df_rw_filepath, param, contact, light, gas, t, s, fs, lw, ms, colormap, plot_legend, save):
    df = pd.read_excel(df_rw_filepath)
    df = df[(df['Contact']==contact)&(df['Light'] == light)&(df['Gas'] == gas)&(df['$T$ (K)']==t)&(df['Sweep']==s)]
    
    param_clean = re.sub(r'\s*\([^)]*\)', '', param)
    match = re.search(r'\$(.*?)\$', param_clean)
    if match:
        param_clean = match.group(1).replace('\\', '')  # remove backslashes from LaTeX
    else:
        param_clean = param_clean.replace('\\', '')
        
    # Check if param is one of the df columns
    if param not in df.columns:
        print(f"Parameter {param} not found in DataFrame columns.")
        return
    
    vgs_values = df['$V_{GS}$ (V)'].unique()
    
    cc = np.linspace(0, 1, len(vgs_values))
    if isinstance(colormap, str):
        if colormap == 'Default':
            colors = [cm.jet(x) for x in cc][::-1]
        else:
            cmap = getattr(cm, colormap)
            colors = [cmap(x) for x in cc][::-1]
    else:
        colors = [colormap(x) for x in cc][::-1]
    
    if len(vgs_values)<4:
        ncol=1
    else:
        ncol=2

    for vgsx, vgs in enumerate(vgs_values):
        df_vgs = df[df['$V_{GS}$ (V)'] == vgs]
        vds_values = df_vgs['$V_{DS}$ (V)'].to_numpy()
        y_values = df_vgs[param].to_numpy()
        plt.plot(vds_values, y_values, label=f'{vgs}', marker='o', markersize=ms, color=colors[vgsx], linewidth=lw)

    plt.xlabel('$V_{DS}$ (V)', fontsize=fs)
    plt.ylabel(param, fontsize=fs)
    plt.tick_params(labelsize=fs)
    plt.grid()
    
    '''
    if param == '$\mu_{RW}$ (cm$^2$/Vs)': y_min, y_max = -6.709766748166008*1e-5, 9.487361364982162*1e-5
    elif param == '$R_c$ ($\Omega$)': y_min, y_max = -93720574457.10103, 411609273715.8159
    elif param == '$R_cW$ (k$\Omega$$\mu$m)': y_min, y_max = -749764595.6568081, 3292874189.7265277
    plt.ylim(y_min, y_max)
    '''
    
    if plot_legend:
        legend = plt.legend(title='$V_{GS}$ (V)', loc='best', fontsize=fs, frameon=True, ncol=ncol)

    title = f'{contact} {t}K step=0.5V {light} {gas} {s}'
    plt.title(title, fontsize=fs)
    
    plt.gca().yaxis.set_major_formatter(FuncFormatter(sci_notation_formatter))

    fig.set_constrained_layout(True)

    if save:
        name = f'{param_clean}_vs_Vds_{contact}_{t}K_{light}_{gas}_{s}'
        folder = f'figures/{param_clean}_vs_Vds'
        # folder = f'../../../Presentations/figures/Parameters'
        os.makedirs(folder, exist_ok=True)
        plt.savefig(f'{folder}/{name}.png', dpi=300, bbox_inches='tight')
        plt.savefig(f'{folder}/{name}.svg', dpi=300, bbox_inches='tight')
        
    y_min, y_max = plt.ylim()
    return y_min, y_max

In [24]:
# Test

df_rw_filepath = 'df_Rc_muRW.xlsx'
param = '$\mu_{RW}$ (cm$^2$/Vs)'
contact = 'Ti-Pd'
light, gas = 'DARK', 'VAC'
t = 298
s ='backward'
fs = 14
lw = 1
ms = 5
colormap = 'jet'
plot_legend = True
save = True

fig = plt.figure(dpi=300)
plot_RW_param_vs_Vds(df_rw_filepath, param, contact, light, gas, t, s, fs, lw, ms, colormap, plot_legend, save)

plt.show()

In [68]:
# Plot all parameters for all contacts

df_rw_filepath = 'df_Rc_muRW.xlsx'
params = ['$\mu_{RW}$ (cm$^2$/Vs)','$R_c$ ($\Omega$)','$R_cW$ (k$\Omega$$\mu$m)']
contacts_lst = ['Ti-Pd', 'Ti-Ti']
light, gas = 'DARK', 'VAC'
t = 298
s ='backward'
fs = 14
lw = 1
ms = 5
colormap = 'jet'
plot_legend = True
save = True

for param in params:
    print(param)
    maxs, mins = [], []
    for contact in contacts_lst:
        fig = plt.figure(dpi=300)
        y_min, y_max = plot_RW_param_vs_Vds(df_rw_filepath, param, contact, light, gas, t, s, fs, lw, ms, colormap, plot_legend, save)
        mins.append(y_min)
        maxs.append(y_max)
    print(min(mins), max(maxs))
plt.show()

$\mu_{RW}$ (cm$^2$/Vs)
-6.709766748166008e-05 9.487361364982162e-05
$R_c$ ($\Omega$)
-93720574457.10103 411609273715.8159
$R_cW$ (k$\Omega$$\mu$m)
-749764595.6568081 3292874189.7265277


### Other parameters

#### Plot either one or all parameters vs $T$, for all combinations of 'Device, Light, Gas, L'

In [26]:
# Plot the selected parameter ('param') vs T @ all Eds values, for ALL unique combinations of 'Device, Light, Gas, L'.

# df_filepath: filepath to dataframe that contains input data; 'param' must be corresponding column name in df
# s: sweep
# fs, lw, ms: plot fontsize, linewidth and markersize
# save: True to save plot, False not to

# -------------------------------------

def plot_param_vs_T(df_filepath, param, s, fs, lw, ms, save):

    df = pd.read_excel(df_filepath)
    
    param_clean = re.sub(r'\s*\([^)]*\)', '', param) # removes '(...)' (to remove units)
    match = re.search(r'\$(.*?)\$', param_clean) # takes what is in '$...$'
    if match:
        param_clean = match.group(1).replace('\\', '') # removes latex '\'
    else:
        param_clean = param_clean.replace('\\', '')
        
        
    # Check if param is one of the df columns
    if param not in df.columns:
        print(f"Parameter '{param}' not found in DataFrame columns.")
        return
    
    df = df[df['Sweep'] == s]
    grouping_cols = ['Device', 'Light', 'Gas', '$L$ (nm)']
    grouped = df.groupby(grouping_cols)
    
    for group_keys, group_df in grouped:
        if group_df['$T$ (K)'].nunique() < 2:
            continue
            
        fig = plt.figure(dpi=300)
        device, light, gas, l = group_keys
        group_df_Eds = group_df.groupby('$E_{DS}$ (V/nm)')
        cc = np.linspace(0, 1, len(group_df_Eds))
        match = re.search(r'^([A-Z])(\d)', device)
        dev_letter = match.group(1)
        dev_number = match.group(2)
        contact = contacts[dev_letter]
        
        for i, (Eds, Eds_df) in enumerate(group_df_Eds):
            ccx = cc[i]
            col_Eds = cm.get_cmap('jet_r')(ccx)
            Eds_df_sorted = Eds_df.sort_values('$T$ (K)') # sort by T for plot (ascending by default)
            
            # Filter out rows where param is NaN or empty
            Eds_df_valid = Eds_df_sorted.dropna(subset=[param])
            T_valid = Eds_df_valid['$T$ (K)']
            param_valid = Eds_df_valid[param]
            
            T_plot, param_plot = [], []
            for ti, pi in zip(T_valid, param_valid):
                if ti >= 255:
                    T_plot.append(ti)
                    param_plot.append(pi)
                    
            plt.plot(T_plot, param_plot, label=f'{Eds}', linewidth=lw, marker='o', markersize=ms, color=col_Eds)
        
        plt.xlabel('$T$ (K)', fontsize=fs)
        plt.ylabel(param, fontsize=fs)
        plt.tick_params(labelsize=fs)
        plt.grid()
        
        if len(group_df_Eds)<4:
            ncol=1
        else:
            ncol=2
        legend = plt.legend(title='$E_{DS}$ (V/nm)', loc='best', fontsize=fs, frameon=True, ncol=ncol)
        
        title = f'{device} ({contact}) {l}nm step=0.5V {light} {gas}'
        name = f'{param_clean}_vs_T_{device}_{l}nm_{light}_{gas}'
        if param != '$N_{it,\\Delta}$ (cm$^{-2}$)':
            title += f' {s}'
            name += f'_{s}'
            
        plt.title(title, fontsize=fs)
        
        plt.gca().yaxis.set_major_formatter(FuncFormatter(sci_notation_formatter))
        fig.set_constrained_layout(True)
        
        if save:
            folder = f'figures/{param_clean}_vs_T'
            os.makedirs(folder, exist_ok=True)
            plt.savefig(f'{folder}/{name}.png', dpi=300, bbox_inches='tight')
            plt.savefig(f'{folder}/{name}.svg', dpi=300, bbox_inches='tight')

In [9]:
# Test

df_filepath = 'df_FET.xlsx'
param = '$N_{it,SS}$ (cm$^{-2}$)'
fs = 14
lw = 1
ms = 5
save = True

plot_param_vs_T(df_filepath, param, 'backward', fs=fs, lw=lw, ms=ms, save=save)
plot_param_vs_T(df_filepath, param, 'forward', fs=fs, lw=lw, ms=ms, save=save)

In [54]:
# Plot all columns starting from '$I_{DS,on}$ (nA)' to the last of df_FET

df_filepath = 'df_FET.xlsx'
df_FET = pd.read_excel(df_filepath)
start_col = '$I_{DS,on}$ (nA)'
fs = 14
lw = 1
ms = 1.5
save = True
# -------------------------------------

# Get list of columns 
col_index = df_FET.columns.get_loc(start_col)
end_col = 'ratio'
endcol_indx = df_FET.columns.get_loc(end_col)
params_to_plot = df_FET.columns[col_index:endcol_indx]

# Plot all
for param in params_to_plot:
    if param == '$N_{it,\\Delta}$ (cm$^{-2}$)':
        plot_param_vs_T(df_filepath, param, 'backward', fs=fs, lw=lw, ms=ms, save=save)
    else:
        for sweep in ['backward','forward']:
            plot_param_vs_T(df_filepath, param, s=sweep, fs=fs, lw=lw, ms=ms, save=save)
            
plt.show()

#### Plot selected parameter for ONE specific combination

In [44]:
# Plot the selected parameter ('param') vs T @ all Eds values, for the ONE selected unique combination of 'Device, Light, Gas, L, Sweep'.

# filepath: filepath to dataframe that contains input data; 'param' must be corresponding column name in dataframe
# dev, light, gas, l, s: device, light, gas, length, sweep
# s_write: show sweep in plot title and filename
# fs, lw, ms, colormap: plot fontsize, linewidth, markersize and colormap
# plot_legend: True to show legend, False not to
# save: True to save plot, False not to

# -------------------------------------

def plot_param_vs_T_1(df_filepath, param, dev, light, gas, l, s, fs, lw, ms, colormap, plot_legend, save):
    df = pd.read_excel(df_filepath)
    
    match = re.search(r'^([A-Z])(\d)', dev)
    dev_letter = match.group(1)
    dev_number = match.group(2)
    contact = contacts[dev_letter]
    
    param_clean = re.sub(r'\s*\([^)]*\)', '', param)
    match = re.search(r'\$(.*?)\$', param_clean)
    if match:
        param_clean = match.group(1).replace('\\', '')  # remove backslashes from LaTeX
    else:
        param_clean = param_clean.replace('\\', '')
        
    # Check if param is one of the df columns
    if param not in df.columns:
        print(f"Parameter '{param}' not found in DataFrame columns.")
        return
    
    mask = (df['Sweep'] == s) & (df['Light'] == light) & (df['Gas'] == gas) & (df['Device'] == dev) & (df['$L$ (nm)'] == l)
    df = df[mask]
    
    if df['$T$ (K)'].nunique() < 2:
        print('Not enough temperatures.')
        return

    df_Eds = df.groupby('$E_{DS}$ (V/nm)')
    
    cc = np.linspace(0, 1, len(df_Eds))
    
    if isinstance(colormap, str):
        if colormap == 'Default':
            colors = [cm.jet(x) for x in cc][::-1]
        else:
            cmap = getattr(cm, colormap)
            colors = [cmap(x) for x in cc][::-1]
    else:
        colors = [colormap(x) for x in cc][::-1]

    for i, (Eds, Eds_df) in enumerate(df_Eds):
        Eds_df_sorted = Eds_df.sort_values('$T$ (K)') # sort by T for plot (ascending by default)

        # Filter out rows where param is NaN or empty
        Eds_df_valid = Eds_df_sorted.dropna(subset=[param])
        T_valid = Eds_df_valid['$T$ (K)']
        param_valid = Eds_df_valid[param]

        T_plot, param_plot = [], []
        for ti, pi in zip(T_valid, param_valid):
            if ti >= 255:
                T_plot.append(ti)
                param_plot.append(pi)
        
        plt.plot(T_plot, param_plot, label=f'{Eds}', marker='o', markersize=ms, color=colors[i], linewidth=lw)
    
    plt.xlabel('$T$ (K)', fontsize=fs)
    plt.ylabel(param, fontsize=fs)
    plt.tick_params(labelsize=fs)
    plt.grid()
    
    if param == '$SS_{min}$ (V/decade)':
        ymin,ymax = plt.ylim()
        y_min, y_max = ymin, min(ymax,20)
        plt.ylim(y_min, y_max)

    if plot_legend:
        if len(df_Eds)<4:
            ncol=1
        else:
            ncol=2
        legend = plt.legend(title='$E_{DS}$ (V/nm)', loc='best', fontsize=fs, frameon=True, ncol=ncol)

    title = f'{dev} ({contact}) {l}nm step=0.5V {light} {gas}'
    name = f'{param_clean}_vs_T_{dev}_{l}nm_{light}_{gas}'
    if param != '$N_{it,\\Delta}$ (cm$^{-2}$)':
        title += f' {s}'
        name += f'_{s}'

    plt.title(title, fontsize=fs)
    
    plt.gca().yaxis.set_major_formatter(FuncFormatter(sci_notation_formatter))

    fig.set_constrained_layout(True)

    if save:
        folder = f'figures/{param_clean}_vs_T'
        os.makedirs(folder, exist_ok=True)
        plt.savefig(f'{folder}/{name}.png', dpi=300, bbox_inches='tight')
        plt.savefig(f'{folder}/{name}.svg', dpi=300, bbox_inches='tight')
    
    if param != '$SS_{min}$ (V/decade)':
        y_min, y_max = plt.ylim()
    return y_min, y_max

In [48]:
# Test

fig = plt.figure(dpi=300)
plot_param_vs_T_1(df_filepath='df_FET.xlsx', param='$I_{DS,off}$ (nA)', dev='H1', light='DARK', gas='VAC', l=500, s='forward', fs=14, lw=1, ms=1.5, colormap='jet', plot_legend=True, save=False)

(-0.0008651847331240156, 0.018335947551374382)

### Box plot

In [80]:
def plot_param_vs_L_alldevs_boxplot(df_filepath, param, Eds, t, s, contact, light, gas, fs, color, save):
    df = pd.read_excel(df_filepath)
    
    param_clean = re.sub(r'\s*\([^)]*\)', '', param)
    match = re.search(r'\$(.*?)\$', param_clean)
    if match:
        param_clean = match.group(1).replace('\\', '')  # remove backslashes from LaTeX
    else:
        param_clean = param_clean.replace('\\', '')
        
    # Check if param is one of the df columns
    if param not in df.columns:
        print(f"Parameter '{param}' not found in DataFrame columns.")
        return
    
    df = df[df['Device'].str[0].map(contacts) == contact]
    mask = (df['Sweep'] == s)  & (df['Light'] == light) & (df['Gas'] == gas) & (df['$E_{DS}$ (V/nm)'] == Eds) & (df['$T$ (K)'] == t)
    df = df[mask]
    
    df = df.sort_values('$L$ (nm)')
    df_groupedby_l = df.groupby('$L$ (nm)')
    lengths = list(df_groupedby_l.groups.keys())
    data = []
    
    for i, (l, df_l) in enumerate(df_groupedby_l):
        # Filter out rows where param is NaN or empty
        df_l_valid = df_l.dropna(subset=[param])
        param_data = df_l_valid[param]
        data.append(param_data)
    
    bp = plt.boxplot(data, labels=lengths, patch_artist=True)

    if color!='Default':
        for patch in bp['boxes']:
            patch.set_facecolor(color)

        c = mcolors.to_rgb(color)
        amount = 0.4
        darkened_color = tuple((1 - amount)*comp + amount*0 for comp in c)
        for median in bp['medians']:
            median.set(color=darkened_color)
    
    plt.xlabel('$L$ (nm)', fontsize=fs)
    plt.ylabel(param, fontsize=fs)
    plt.tick_params(labelsize=fs)
    plt.grid()
    
    '''
    if param == '$V_{th}$ (V)':
        ymin, ymax = -77.38092084090121, 393.62982136728004
        plt.ylim(ymin,ymax)
    elif param == '$\\mu_{lin}$ (cm$^2$/Vs)':
        ymin, ymax = -5.6286503791483e-06, 0.00014266880775213025
        plt.ylim(ymin,ymax)
    elif param == '$\\mu_{sat}$ (cm$^2$/Vs)':
        ymin, ymax = -0.05500000000000001, 0.05500000000000001
        plt.ylim(ymin,ymax)
    '''

    title = f'{contact} {Eds}V/nm {t}K step=0.5V {light} {gas}'
    name = f'{param_clean}_vs_L_{contact}_{Eds}Vpernm_{t}K_{light}_{gas}'
    if param != '$N_{it,\\Delta}$ (cm$^{-2}$)':
        title += f' {s}'
        name += f'_{s}'
    
    '''
    name+='_unifiedylims'
    '''
    
    plt.title(title, fontsize=fs)
    
    plt.gca().yaxis.set_major_formatter(FuncFormatter(sci_notation_formatter))
    fig.set_constrained_layout(True)

    if save:
        folder = f'figures/{param_clean}_vs_L_boxplot'
        os.makedirs(folder, exist_ok=True)
        filepath = f'{folder}/{name}'
        plt.savefig(f'{filepath}.png', dpi=300, bbox_inches='tight')
        plt.savefig(f'{filepath}.svg', dpi=300, bbox_inches='tight')
    else:
        filepath = None
        
    y_min, y_max = plt.ylim()
    
    return y_min, y_max, filepath

In [81]:
# Test

df_filepath = 'df_FET.xlsx'
param = '$V_{th}$ (V)'
Eds = 0.005
t = 298
s = 'backward'
contact = 'Ti-Ti'
light, gas = 'DARK', 'VAC'
fs = 14
save = True
color = 'Default'

fig = plt.figure(dpi=300)
plot_param_vs_L_alldevs_boxplot(df_filepath, param, Eds, t, s, contact, light, gas, fs, color, save)

(-17.376996643679494,
 -5.960522029491072,
 'figures/V_{th}_vs_L_boxplot/V_{th}_vs_L_Ti-Ti_0.005Vpernm_298K_DARK_VAC_backward')

In [49]:
# Plot all columns starting from '$I_{DS,on}$ (nA)' to the last of df_FET, @Eds=0.005V/nm

df_filepath = 'df_FET.xlsx'
df_FET = pd.read_excel(df_filepath)
start_col = '$I_{DS,on}$ (nA)'
Eds = 0.005
t = 298
contact = 'Ti-Ti'
light, gas = 'DARK', 'VAC'
fs = 14
save = True
color = 'Default'
# -------------------------------------

# Get list of columns 
col_index = df_FET.columns.get_loc(start_col)
params_to_plot = df_FET.columns[col_index:]

# Plot all
for param in params_to_plot:
    if param == '$N_{it,\\Delta}$ (cm$^{-2}$)':
        fig = plt.figure(dpi=300)
        plot_param_vs_L_alldevs_boxplot(df_filepath, param, Eds, t, 'backward', contact, light, gas, fs, color, save)
    else:
        for sweep in ['backward','forward']:
            fig = plt.figure(dpi=300)
            plot_param_vs_L_alldevs_boxplot(df_filepath, param, Eds, t, sweep, contact, light, gas, fs, color, save)
            
plt.show()

In [32]:
# Boxplots for one parameter, for all Eds 'standard' values and for both sweeps and contacts

df_filepath = 'df_FET.xlsx'

param_lst = ['$V_{th}$ (V)', '$\\mu_{lin}$ (cm$^2$/Vs)', '$\\mu_{sat}$ (cm$^2$/Vs)']
param = param_lst[0]

Eds_lst = [-0.02, -0.01, -0.005, 0.005, 0.01, 0.02]
t = 298
contacts_lst = ['Ti-Ti', 'Ti-Pd']
light, gas = 'DARK', 'VAC'
fs = 14
save = True
color = 'Default'

mins,maxs=[],[]

for s in ['backward','forward']:
    for contact in contacts_lst:
        for Eds in Eds_lst:
            fig = plt.figure(dpi=300)
            y_min, y_max, _ = plot_param_vs_L_alldevs_boxplot(df_filepath, param, Eds, t, s, contact, light, gas, fs, color, save)
            mins.append(y_min)
            maxs.append(y_max)
            
print(min(mins), max(maxs))

ymin, ymax:

Vth: -77.38092084090121, 393.62982136728004

mu_lin: -5.6286503791483e-06, 0.00014266880775213025

mu_sat: -0.05500000000000001, 0.05500000000000001

In [82]:
# Put together all boxplots for one parameter in a single image

from PIL import Image
import math

Eds_lst = [-0.02, -0.01, -0.005, 0.005, 0.01, 0.02]
t = 298
contacts_lst = ['Ti-Ti', 'Ti-Pd']
light, gas = 'DARK', 'VAC'

param_lst = ['$V_{th}$ (V)', '$\\mu_{lin}$ (cm$^2$/Vs)', '$\\mu_{sat}$ (cm$^2$/Vs)']
param = param_lst[2]

#-----------------------------------

param_clean = re.sub(r'\s*\([^)]*\)', '', param)
match = re.search(r'\$(.*?)\$', param_clean)
if match:
    param_clean = match.group(1).replace('\\', '')  # remove backslashes from LaTeX
else:
    param_clean = param_clean.replace('\\', '')

folder = f'figures/{param_clean}_vs_L_boxplot'

image_grid = []
for contact in contacts_lst:
    for s in ['forward', 'backward']:
        row = []
        for Eds in Eds_lst:
            name = f'{param_clean}_vs_L_{contact}_{Eds}Vpernm_{t}K_{light}_{gas}'
            if param != '$N_{it,\\Delta}$ (cm$^{-2}$)':
                name += f'_{s}'
            # name+='_unifiedylims'
            filepath = f'{folder}/{name}.png'
            row.append(filepath)
        image_grid.append(row)

        
    
# Resize images

target_width = 600
target_height = 400

def resize_keep_ratio(im, target_w, target_h):
    ratio = min(target_w / im.width, target_h / im.height)
    new_size = (int(im.width * ratio), int(im.height * ratio))
    return im.resize(new_size, Image.ANTIALIAS)
    # Image.ANTIALIAS is used to smooth the image during resizing (for quality)

images_resized = []
for row in image_grid:
    resized_row = []
    for path in row:
        if os.path.exists(path):
            im = Image.open(path)
            im_resized = resize_keep_ratio(im, target_width, target_height)
            resized_row.append(im_resized)
        else:
            print(f"Missing: {path}")
            # Add blank slot instead of image if file is missing
            placeholder = Image.new('RGBA', (target_width, target_height), (255, 255, 255, 255)) # solid white rectangle
            resized_row.append(placeholder)
    images_resized.append(resized_row)

    
# Create mosaic

cols = len(Eds_lst)
rows = len(images_resized)
mosaic_width = cols * target_width
mosaic_height = rows * target_height

mosaic = Image.new('RGBA', (mosaic_width, mosaic_height), (255, 255, 255, 255))

for row_idx, row in enumerate(images_resized):
    for col_idx, img in enumerate(row):
        x = col_idx * target_width + (target_width - img.width) // 2
        y = row_idx * target_height + (target_height - img.height) // 2
        if img.mode == 'RGBA':
            mosaic.paste(img, (x, y), img)
        else:
            mosaic.paste(img, (x, y))

            
# Save

output_filename = f'boxplot_mosaic_{param_clean}' # _unifiedylims
output_filepath = f'{folder}/{output_filename}.png'
mosaic.save(output_filepath)
print(f"Mosaic saved with filepath: {output_filepath}")

Mosaic saved with filepath: figures/mu_{sat}_vs_L_boxplot/boxplot_mosaic_mu_{sat}.png
